In [ ]:
from __future__ import print_function, division
import scipy
from keras.datasets import mnist
#from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, concatenate
from keras.layers import BatchNormalization, Activation, ZeroPadding2D, Add, Average
from keras.layers.advanced_activations import PReLU, LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.applications import VGG19
from keras.models import Sequential, Model
from keras.optimizers import Adam
import datetime
import matplotlib.pyplot as plt
import sys
#from data_loader import DataLoader
import numpy as np
import os
import keras.backend as K
from keras.layers import Input, Activation, Add
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.core import Lambda
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras import objectives
from keras import backend as K
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import Input, merge, Dense, Lambda
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.normalization import BatchNormalization
from keras.layers.core import Activation, Flatten, Reshape, Dropout
from keras.layers.convolutional import Convolution2D, Deconvolution2D, UpSampling2D,AtrousConv2D
import numpy as np
from keras.layers.convolutional import UpSampling2D, Conv2D

In [ ]:
import tensorflow as tf

from keras.models import Model
from keras.engine import InputSpec
from keras.engine.topology import Layer
from keras.layers import Input, Conv2D, Activation, BatchNormalization
from keras.layers.merge import Add
from keras.utils import conv_utils
from keras.layers.core import Dropout
import keras.backend as K

In [ ]:

from keras.layers import Dense
from keras.layers.core import Activation
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import UpSampling2D
from keras.layers.core import Flatten
from keras.layers import Input
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.models import Model
from keras.layers.advanced_activations import LeakyReLU, PReLU
from keras.layers import add

Model Building

In [17]:
ngf = 64
input_nc = 3
output_nc = 3
input_shape_generator = (256, 256, input_nc)
n_blocks_gen = 9
n_downsampling = 1
def streamdense(inputs): #MFAB block
    f=64
    xin = Conv2D(filters=f, kernel_size=(3,3), padding='same')(inputs)
    x1 = Conv2D(filters=f, kernel_size=(3,3), padding='same')(inputs)
    x1= LeakyReLU(alpha=0.2)(x1)

    x4 = Conv2D(filters=f, kernel_size=(7,7), padding='same')(inputs)
    x4 = LeakyReLU(alpha=0.2)(x4)

    x5 = Conv2D(filters=f, kernel_size=(9,9), padding='same')(inputs)
    x5 = LeakyReLU(alpha=0.2)(x5)
    x6 = Average()([x1,x4,x5])
    x5 = Add()([xin,x6])
    return x5

def res_block(layer):
    d = Conv2D(filters=64, kernel_size=3, strides=1, padding='same')(layer)
    d = BatchNormalization(momentum=0.8)(d)
    d = LeakyReLU(alpha=0.2)(d)
    d = Add()([d, layer])
    return d

def deconv2d(layer_input):
    u = UpSampling2D(size=2)(layer_input)
    u = Conv2D(256, kernel_size=3, strides=1, padding='same')(u)
    u = Activation('relu')(u)
    return u


def build_generator1():
    image_shape=(256,256,3)
    inputs = Input(shape=image_shape)
    x1=inputs
    
    x1 = Conv2D(filters=ngf, kernel_size=(7, 7), padding='same')(x1)
    x1 = BatchNormalization()(x1)
    x1 = Activation('relu')(x1)
    x=x1

    for t1 in range(5):
        #f=64*(2**t1)
        x=streamdense(x)
    mult = 2**n_downsampling

    for i in range(n_blocks_gen):
        x = res_block(x)

    for i in range(n_downsampling):
        mult = 2**(n_downsampling - i)
        x = Conv2D(filters=int(ngf * mult / 2), kernel_size=(3, 3), padding='same')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
    x=concatenate([x,x1])

    
    x = Conv2D(filters=256, kernel_size=(7, 7), padding='same')(x)
    x = Activation('tanh')(x)

    outputs = Lambda(lambda z: z/2)(x)
    #outputs =deconv2d(outputs). #Depending upon the scaling of super-resolution deconv layers can be increased or decreased
    outputs = deconv2d(outputs)
    outputs = Conv2D(filters=3, kernel_size=(3, 3), padding='same')(outputs)
    model = Model(inputs=inputs, outputs=outputs, name='Generator')
    return model


In [ ]:
build_generator1().summary()

In [ ]:
    df=64
    def build_discriminator():

        def d_block(layer_input, filters, strides=1, bn=False):
            """Discriminator layer"""
            d = Conv2D(filters, kernel_size=3, strides=strides, padding='same')(layer_input)
            d = LeakyReLU(alpha=0.2)(d)
            if bn:
                d = BatchNormalization(momentum=0.8)(d)
            return d

        # Input img
        d0 = Input(shape=(512,512,3))

        d1 = d_block(d0, df, bn=False)
        d2 = d_block(d1, df, strides=2)
        d3 = d_block(d2, df*2)
        d4 = d_block(d3, df*2, strides=2)
        d5 = d_block(d4, df*4)
        d6 = d_block(d5, df*4, strides=2)
        d7 = d_block(d6, df*8)
        d8 = d_block(d7, df*8, strides=2)

        d9 = Dense(df*16)(d8)
        d10 = LeakyReLU(alpha=0.2)(d9)
        d11 = Flatten()(d10)
        validity = Dense(1, activation='sigmoid')(d11)

        return Model(d0, validity)

In [ ]:
build_discriminator().summary()

Dataset

In [ ]:
def saveModels(epoch,generator):
    generator.save('/content/gdrive/My Drive/denoisegan/28jan_contd..h5')
    #build_discriminator().save('/home/vengulab/sourya/sr_gan/gan_discriminator_epoch_%d.h5' % epoch)

In [ ]:
y_train1=np.load('/content/gdrive/My Drive/denoisegan/messidorhi.npy')
y_train=y_train1[0:698,:,:,:]
y_train.shape

In [ ]:

x_train1=np.load('/content/gdrive/My Drive/denoisegan/dataset/messidor_motion_blu_256.npy')
y= y_train1[0:698,:,:,:]
x= x_train1[0:698,:,:,:]

In [ ]:
ytest= y_train1[699:798,:,:,:]
xtest= x_train1[699:798,:,:,:]

Loss Functions and Training

In [ ]:
def build_vgg():
    vgg = VGG19(weights="imagenet")
    vgg.outputs = [vgg.layers[9].output]
    img = Input(shape=(512,512,3))
    img_features = vgg(img)
    return Model(img, img_features)

In [ ]:
def loss_G(y_true, y_pred):
        L1_distance = K.mean(K.abs((y_true-y_pred)**2))
        x = y_true
        y = y_pred
        ave_x = K.mean(x)
        ave_y = K.mean(y)
        var_x = K.var(x)
        var_y = K.var(y)
        covariance = K.mean(x*y) - ave_x*ave_y
        c1 = 0.01**2
        c2 = 0.03**2
        ssim = (2*ave_y*ave_x+c1)*(2*covariance+c2)
        ssim = ssim/((K.pow(ave_x,2)+K.pow(ave_y,2)+c1) * (var_x+var_y+c2))
        dssim = 1 - ssim
        return dssim

In [ ]:
def train(epochs,batch_size):
    g_loss_lo=100
    #y_train1=np.load('/content/gdrive/My Drive/denoisegan/messidorhi.npy')
    #x_train1=np.load('/content/gdrive/My Drive/denoisegan/messidor_motion_blu_256.npy')
    y_train=y
    x_train=x
    lr=0.0001
    optimizer = Adam(0.0001, beta_1=0.9)
    vgg = build_vgg()
    vgg.trainable = False
    vgg.compile(loss='mse', optimizer=optimizer,metrics=['accuracy'])
    gf=64
    df=64
    discriminator = build_discriminator()
    discriminator.compile(loss='mse',optimizer=optimizer,metrics=['accuracy'])
    generator = build_generator1()
    img_hr = Input(shape=(512,512,3))
    img_lr = Input(shape=(256,256,3))
    fake_hr = generator(img_lr)
    fake_features = vgg(fake_hr)
    discriminator.trainable = False
    validity = discriminator(fake_hr)
    combined = Model([img_lr, img_hr], [validity, fake_features])
    combined.compile(loss=['binary_crossentropy', loss_G], loss_weights=[1e-4, 1], optimizer=optimizer)
    permutated_indexes = np.random.permutation(x_train.shape[0])
    for epoch in range(epochs):
        d_loss=[]
        g_loss=[]

        for index in range(int(x_train.shape[0] / batch_size)):
            batch_indexes = permutated_indexes[index*batch_size:(index+1)*batch_size]
            imgs_hr = y_train[batch_indexes]
            imgs_lr =x_train[batch_indexes]
            #print(imgs_lr.shape)
            fake_hr = generator.predict(imgs_lr)
            valid = np.ones((batch_size))
            fake = np.zeros((batch_size))
            discriminator.compile
            d_loss_real = discriminator.train_on_batch(imgs_hr, valid)
            d_loss_fake = discriminator.train_on_batch(fake_hr, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
            image_features = vgg.predict(imgs_hr)
            g_loss = combined.train_on_batch([imgs_lr, imgs_hr], [valid, image_features])
            #print(fake_hr.shape)
            #g_loss_lo=g_loss[]
        if (g_loss[0]<g_loss_lo):
            #plt.figure()
            #plt.imshow(imgs_hr[0])
            g_loss_lo=g_loss[0]
            #plt.figaure()
            #plt.imshow(fake_hr[0])
            saveModels(epoch,generator)
            
        print ("%d dloss: %s gloss:%s" % (epoch, d_loss[0],g_loss[0]))
    return generator,combined

In [ ]:
def loss_G1(y_true, y_pred):
        L1_distance = np.mean(np.abs((y_true-y_pred)**2))
        x = y_true
        y = y_pred
        ave_x = np.mean(x)
        ave_y = np.mean(y)
        var_x = np.var(x)
        var_y = np.var(y)
        covariance = np.mean(x*y) - ave_x*ave_y
        c1 = 0.01**2
        c2 = 0.03**2
        ssim = (2*ave_y*ave_x+c1)*(2*covariance+c2)
        ssim = ssim/((pow(ave_x,2)+pow(ave_y,2)+c1) * (var_x+var_y+c2))
        #dssim = 1 - ssim
        return ssim

In [ ]:
train(epochs=60, batch_size=1)

In [ ]:
from keras.models import load_model
a1=load_model('/content/gdrive/My Drive/denoisegan/28jan.h5')
#a2=load_model('/content/gdrive/My Drive/denoisegan/srgan_motion_256_trainingonly.h5')"model.h5"

In [ ]:
a1.summary()

In [ ]:
from keras.utils.vis_utils import plot_model

In [ ]:
plot_model(a1, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
a1.compile
#a2.compile


Testing and Image Quality Metric Calculation


In [ ]:
a1_data=np.load('/content/gdrive/My Drive/denoisegan/messidor_motion_blu_256.npy')
a2_data=messidor_clear_256
a1test= a1_data[699:798,:,:,:]
a2test= a2_data[699:798,:,:,:]

In [ ]:
import cv2
cv2.Laplacian(plt.imread('/content/gdrive/My Drive/denoisegan/testimages256/blurry1.png'), cv2.CV_64F).var()

In [ ]:
data=xtest[28]
#resized = cv2.resize(a1, (256,256), interpolation = cv2.INTER_AREA)
lr2= data.reshape((1,256,256,3))
x_hr=a1.predict(lr2)
x_hr_1=x_hr[0]
#x_hr_final=a2.predict(x_hr_1)
plt.imshow(x_hr_1[100:300,100:300])
plt.figure()
plt.imshow(ytest[28][100:300,100:300])
plt.figure()
plt.imshow(xtest[28][50:150,50:150])
plt.figure()
#crop_img = x_hr_1[50:150,50:150]
#plt.imshow(crop_img)
#print(psnr(ytest[50],x_hr_final[0]))

In [ ]:
save_img('/content/gdrive/My Drive/denoisegan/tes3.png',x_hr_final[0])

In [ ]:
import cv2
p_test=[]
p_cv=[]
for t1 in range(len(xtest)):
    x1=xtest[t1]
    lr2= x1.reshape((1,256,256,3))
    #x1.reshape((1,256,256,3))
    x_hr=a1.predict(lr2)
    p_test.append(psnr(ytest[t1],x_hr[0]))
    print(psnr(ytest[t1],x_hr[0]))
    #p_cv.append(psnr(ytest[t1],cv2.resize(xtest[t1],(512,512))))

In [ ]:
s_test=[]
s_cv=[]
for t1 in range(len(xtest)):
    x1=xtest[t1]
    lr2= x1.reshape((1,256,256,3))
    #x1.reshape((1,256,256,3))
    x_hr=a1.predict(lr2)
    s_test.append(loss_G1(ytest[t1],x_hr[0]))
    #print(ssim(ytest[t1],x_hr[0]))
    #p_cv.append(psnr(ytest[t1],cv2.resize(xtest[t1],(512,512))))

In [ ]:
sum(s_test)/len(s_test)

0.997019186911804

In [ ]:
import math
def psnr(img1, img2):
    mse = np.mean( (img1 - img2) ** 2 )
    PIXEL_MAX = 255.0
    return 10 * math.log10(PIXEL_MAX / math.sqrt(mse))

In [ ]:
import cv2

In [ ]:
#y_train=np.load('/content/gdrive/My Drive/denoisegan/messidorhi.npy')
p=[]
for t1 in range(len(b)):
    a1=lr[t1]
    #lr2= a1.reshape((1,128,128,3))
    #hr_p=a.predict(lr2)
    psnr1=psnr(cv2.resize(a1,(512,512)),b[t1])
    p.append(psnr1)


In [ ]:
sum(p)/len(p)